## Helper functions

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import numpy as np

from others.implementations import *
import torch
from model import *

In [26]:
def generate_model(shallow_hidden_channels, deep_hidden_channels):
    return Sequential(
        Conv2d(3, shallow_hidden_channels, kernel_size=2, stride=2, padding=0),
        ReLU(),
        Conv2d(shallow_hidden_channels, deep_hidden_channels, kernel_size=2, stride=2, padding=0),
        ReLU(),
        NearestUpsampling(scale_factor=2),
        Conv2d(deep_hidden_channels, shallow_hidden_channels, kernel_size=3, stride=1, padding=1),
        ReLU(),
        NearestUpsampling(scale_factor=2),
        Conv2d(shallow_hidden_channels, 3, kernel_size=3, stride=1, padding=1),
        Sigmoid()
        )

def test_and_train(batch_size, hidden_channels, lr, momentum, nesterov, epochs):
    network = generate_model(shallow_hidden_channels=hidden_channels[0], deep_hidden_channels=hidden_channels[1])
    optimizer = SGD(network.param(), lr=lr, momentum=momentum, nesterov=nesterov)

    # Instantiate model and replace network and optimizer
    m = Model()
    m.model = network
    m.optimizer = optimizer
    m.batch_size = batch_size

    # Train
    m.train(s1, s2, epochs)

    return compute_psnr(m.predict(t1), t2), m

def sample(tensor1, tensor2, k):
    perm = torch.randperm(tensor1.size(0))
    idx = perm[:k]
    return tensor1[idx], tensor2[idx]

## Loading data

In [27]:
path_train = '../data/train_data.pkl'
path_val = '../data/val_data.pkl'
noisy_imgs_1, noisy_imgs_2 = torch.load(path_train)
noisy_imgs_1, noisy_imgs_2 = noisy_imgs_1.double(), noisy_imgs_2.double()
test, truth = torch.load(path_val)
test, truth = test.double(), truth.double()

## Parameters to tune

In [28]:
# Channels parameter
shallow_hidden_channels = [8, 16, 32, 64, 128]
deep_hidden_channels = [8, 16, 32, 64, 128, 256]
hidden_channels = [(shallow_channel, deep_channel) for shallow_channel in shallow_hidden_channels for deep_channel in deep_hidden_channels if shallow_channel <= deep_channel]

# Optimizer parameters
lrs = np.logspace(-7, -1, 7)
momentums = [0.9]
nesterovs = [True, False]

# Batch sizes
batch_sizes = [5, 10, 20]

## Training

In [29]:
s1, s2 = sample(noisy_imgs_1, noisy_imgs_2, 1000)
t1, t2 = sample(test, truth, 1000)

In [30]:
epochs = 5

results = dict()
for lr in lrs:
    for momentum in momentums:
        for nesterov in nesterovs:
            for hidden_channel in hidden_channels:
                for batch_size in batch_sizes:
                    description = f'lr{lr}_HiddenChannels{hidden_channel}_Batch{batch_size}_Epochs{epochs}_Momentum{momentum}_Nesterov{nesterov}_Sample1000'
                    print("\nSTARTING TRAINING FOR:", description)
                    error, m = test_and_train(batch_size, hidden_channel, lr, momentum, nesterov, epochs)
                    description = f'lr{lr}_HiddenChannels{hidden_channel}_Batch{batch_size}_Epochs{epochs}_Momentum{momentum}_Nesterov{nesterov}_Sample1000_{error}'
                    results[description] = m


STARTING TRAINING FOR: lr1e-07_HiddenChannels(8, 8)_Batch5_Epochs5_Momentum0.9_NesterovTrue_Sample1000
EPOCH 0 --- LOSS 0.04328843472746895
EPOCH 0 --- LOSS 0.07806579561227261
EPOCH 0 --- LOSS 0.0651257995760975
EPOCH 0 --- LOSS 0.09737408798087546
EPOCH 0 --- LOSS 0.07295253862832617
EPOCH 0 --- LOSS 0.0706024427054219
EPOCH 0 --- LOSS 0.059634915342102295
EPOCH 0 --- LOSS 0.10519822535808862
EPOCH 0 --- LOSS 0.07682966778735126
EPOCH 0 --- LOSS 0.08058053370526669
EPOCH 0 --- LOSS 0.05826480142721176
EPOCH 0 --- LOSS 0.07796428915428841
EPOCH 0 --- LOSS 0.08655010850898227
EPOCH 0 --- LOSS 0.05730750195105887
EPOCH 0 --- LOSS 0.04932028422541331
EPOCH 0 --- LOSS 0.07500797113709974
EPOCH 0 --- LOSS 0.07962770086456782
EPOCH 0 --- LOSS 0.059316349104609103
EPOCH 0 --- LOSS 0.10063758877841546
EPOCH 0 --- LOSS 0.06494418714154977
EPOCH 0 --- LOSS 0.09632761496098173
EPOCH 0 --- LOSS 0.08851656516191228
EPOCH 0 --- LOSS 0.05203959334894008
EPOCH 0 --- LOSS 0.08086531315200679
EPOCH 0 

Traceback (most recent call last):
  File "G:\Users\Giovanni\miniconda3\envs\ada\lib\site-packages\IPython\core\interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Giovanni\AppData\Local\Temp\ipykernel_12372\3229998138.py", line 11, in <cell line: 4>
    error, m = test_and_train(batch_size, hidden_channel, lr, momentum, nesterov, epochs)
  File "C:\Users\Giovanni\AppData\Local\Temp\ipykernel_12372\3193474046.py", line 28, in test_and_train
    return compute_psnr(m.predict(t1), t2), m
  File "G:\Giovanni\Università\EPFL\MA2\DL\Projects\deep-learning-project\Miniproject_2\model.py", line 51, in predict
    return (self.model.forward(test_input / 256) * 256).round()
  File "G:\Giovanni\Università\EPFL\MA2\DL\Projects\deep-learning-project\Miniproject_2\others\implementations.py", line 248, in forward
    x = module.forward(x)
  File "G:\Giovanni\Università\EPFL\MA2\DL\Projects\deep-learning-project\Miniproject_2\others\impl

## Final results

In [1]:
results

NameError: name 'results' is not defined

In [23]:
m = Model()
m.train(s1, s2, epochs)

EPOCH 0 --- LOSS 2429434161.2005405
EPOCH 0 --- LOSS 2724892653.0
EPOCH 0 --- LOSS 2580272613.0
EPOCH 0 --- LOSS 2545925880.0
EPOCH 0 --- LOSS 2936724088.0
EPOCH 0 --- LOSS 2365490236.0
EPOCH 0 --- LOSS 2404719814.25
EPOCH 0 --- LOSS 2655381158.0
EPOCH 0 --- LOSS 2709129253.0
EPOCH 0 --- LOSS 2582328629.0
EPOCH 0 --- LOSS 2623651594.0
EPOCH 0 --- LOSS 2777295267.5
EPOCH 0 --- LOSS 2834655888.0
EPOCH 0 --- LOSS 2660729847.0
EPOCH 0 --- LOSS 2644410451.0
EPOCH 0 --- LOSS 2606615400.0
EPOCH 0 --- LOSS 2666495991.25
EPOCH 0 --- LOSS 2531145695.0
EPOCH 0 --- LOSS 2639053727.0
EPOCH 0 --- LOSS 2659015570.75
EPOCH 1 --- LOSS 2407282679.0
EPOCH 1 --- LOSS 2724892653.0
EPOCH 1 --- LOSS 2580272613.0
EPOCH 1 --- LOSS 2545925880.0
EPOCH 1 --- LOSS 2936724088.0
EPOCH 1 --- LOSS 2365490236.0
EPOCH 1 --- LOSS 2404719814.25
EPOCH 1 --- LOSS 2655381158.0
EPOCH 1 --- LOSS 2709129253.0
EPOCH 1 --- LOSS 2582328629.0
EPOCH 1 --- LOSS 2623651594.0
EPOCH 1 --- LOSS 2777295267.5
EPOCH 1 --- LOSS 2834655888.0


KeyboardInterrupt: 